<a href="https://colab.research.google.com/github/lualua0909/DeepLearning-LDS8/blob/main/Week_2/demo_ann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Demo ANN**


In [ ]:
import pandas as pd

In [ ]:

red_wine = pd.read_csv('https://raw.githubusercontent.com/lualua0909/DeepLearning-LDS8/main/Data/red-wine.csv')
red_wine.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


#**Create trainning and validation splits**


In [ ]:
df_train = red_wine.sample(frac=0.7, random_state=0)
df_valid = red_wine.drop(df_train.index)
df_train.head(5)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
1109,10.8,0.470,0.43,2.10,0.171,27.0,66.0,0.99820,3.17,0.76,10.8,6
1032,8.1,0.820,0.00,4.10,0.095,5.0,14.0,0.99854,3.36,0.53,9.6,5
1002,9.1,0.290,0.33,2.05,0.063,13.0,27.0,0.99516,3.26,0.84,11.7,7
487,10.2,0.645,0.36,1.80,0.053,5.0,14.0,0.99820,3.17,0.42,10.0,6
979,12.2,0.450,0.49,1.40,0.075,3.0,6.0,0.99690,3.13,0.63,10.4,5


#**Split featuresa and target**

In [ ]:
X_train = df_train.drop('quality', axis=1)
X_valid = df_valid.drop('quality', axis=1)
y_train = df_train['quality']
y_valid = df_valid['quality']

In [ ]:
X_train.shape

(1119, 11)

In [ ]:
X_valid.shape

(480, 11)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers

#**Cách chọn số node trong hidden layer**

Có một số cách chọn số node, ví dụ:

1. node trong hidden layer = trung bình của số node trong input layer và output layer tương ứng (hoạt động tốt trong 90% các trường hợp)

2. Hoặc node trong hidden layer < node trong input layer * 2

3. Hoặc node trong hidden layer là một giá trị từ node trong input đến node trong output

4. Hoặc node trong hidden layer = 2/3 (node trong input + node trong output)

5. Hoặc có thể là bất cứ giá trị nào (tùy chọn và thử nghiệm, ví dụ: 16, 32, 64, 128, 256, 512...)

In [ ]:
model = keras.Sequential([
                          layers.Dense(6, activation='relu', input_shape=[11]),
                          layers.Dense(6, activation='relu', input_shape=[11]),
                          layers.Dense(1)
])

**Biên dịch model**

In [ ]:
model.compile(optimizer='adam', loss='mae')